In [1]:
from keras import models
from keras import layers
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir ='/notebooks/sharedfolder/data/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
## now, let's use a pretrained model - VGG16
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [4]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [5]:
print('trainable weights before freezing conv_base:', len(model.trainable_weights))
conv_base.trainable = False
print('trainable weights after freezing conv_base:', len(model.trainable_weights))

trainable weights before freezing conv_base: 30
trainable weights after freezing conv_base: 4


In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
 - 84s - loss: 0.5822 - acc: 0.7005 - val_loss: 0.4358 - val_acc: 0.8320
Epoch 2/30
 - 26s - loss: 0.4668 - acc: 0.7935 - val_loss: 0.3616 - val_acc: 0.8630
Epoch 3/30
 - 27s - loss: 0.4194 - acc: 0.8200 - val_loss: 0.3230 - val_acc: 0.8820
Epoch 4/30
 - 27s - loss: 0.4024 - acc: 0.8270 - val_loss: 0.3292 - val_acc: 0.8600
Epoch 5/30
 - 27s - loss: 0.3787 - acc: 0.8425 - val_loss: 0.2928 - val_acc: 0.8840
Epoch 6/30
 - 27s - loss: 0.3654 - acc: 0.8365 - val_loss: 0.2833 - val_acc: 0.8900
Epoch 7/30
 - 26s - loss: 0.3417 - acc: 0.8555 - val_loss: 0.2750 - val_acc: 0.8900
Epoch 8/30
 - 27s - loss: 0.3544 - acc: 0.8440 - val_loss: 0.2667 - val_acc: 0.8940
Epoch 9/30
 - 27s - loss: 0.3495 - acc: 0.8435 - val_loss: 0.2632 - val_acc: 0.8930
Epoch 10/30
 - 27s - loss: 0.3260 - acc: 0.8575 - val_loss: 0.2604 - val_acc: 0.8950
Epoch 11/30
 - 27s - loss: 0.3324 - acc: 0.8510 - val_loss: 0.2567 - val_ac

In [8]:
## lets fine tune the model.. only freeze layers upto block4_pool and unfreeze the layers after
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [ ]:
conv_base.trainable=True
set_trainable=False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable=False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)            

Epoch 1/100
100/100 [==============================] - 48s 477ms/step - loss: 0.3071 - acc: 0.8670 - val_loss: 0.1471 - val_acc: 0.9440
Epoch 2/100
100/100 [==============================] - 45s 450ms/step - loss: 0.2160 - acc: 0.9115 - val_loss: 0.1380 - val_acc: 0.9390
Epoch 3/100
100/100 [==============================] - 46s 455ms/step - loss: 0.1732 - acc: 0.9305 - val_loss: 0.1784 - val_acc: 0.9400
Epoch 4/100
100/100 [==============================] - 45s 454ms/step - loss: 0.1471 - acc: 0.9440 - val_loss: 0.1245 - val_acc: 0.9570
Epoch 5/100
100/100 [==============================] - 46s 456ms/step - loss: 0.1376 - acc: 0.9420 - val_loss: 0.0955 - val_acc: 0.9670
Epoch 6/100
100/100 [==============================] - 45s 454ms/step - loss: 0.1139 - acc: 0.9570 - val_loss: 0.0841 - val_acc: 0.9700
Epoch 7/100
100/100 [==============================] - 45s 454ms/step - loss: 0.0981 - acc: 0.9595 - val_loss: 0.0949 - val_acc: 0.9590
Epoch 8/100
100/100 [===========================

In [1]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [ ]:
## finally test the model on some test data
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150), batch_size=20, class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc: ', test_acc)